In [1]:
# importing pandas for the dataframes
import pandas as pd
import matplotlib.pyplot as plt
import math as math

# output to notebook
%matplotlib inline

#importing sys to increase the recursion limit, 
import sys
sys.setrecursionlimit(25000)

#importing the higher function reduce for a specific part of the code
from functools import reduce


#the proces ID to get into bigquery, this information cannot be shared since this is an active startup 'Headingon'
projectid = "vast-art-**********"

In [2]:

class acceleration:    
    def looping(self, vehicle, start_row):
        """This is the main function for retrieving all the data from bigquery"""       
        total_list = []
        total_df= pd.DataFrame(total_list)
        partial_list = []
        
        for i in range(2):
            partial_list= self.bigquery(vehicle, start_row)
            partial_df = pd.DataFrame(partial_list)
            total_df = pd.concat([total_df,partial_df], ignore_index=True)
            start_row = 11000 + start_row

        return total_df
            
    
    
    def bigquery (self,vehicle, start_row):
        """specif function which targets the bigquery database and reformats it"""
        car_list = []
        
        end_of_row = start_row + 11000
        
         #the query to bigquery and readjusting the format
        query_df = pd.read_gbq("select * from [headingon_params.speed_obd] where vehicle ='%s' order by timestamp asc limit 11000 offset %s " %(vehicle,start_row), projectid)
        transit_df = query_df.drop(['fullname', 'unit'], axis=1)
        column_indicators= ['timestamp', 'value', 'vehicle']
        main_df = transit_df[column_indicators]
        return self.filter_function(vehicle, 0, 11000, main_df, car_list)
    
    
    def filter_function(self,vehicle_name,row, end, dataframe, car_list):
        """This function filters the data which is pulled by the f(bigquery) and filters out the blocks that will be used
            by f(core_function) """
        #the basecase
        if row >= end:
            return car_list
        else:
            #these two if-statements ensures that right rows are identified
            if dataframe.iloc[row,1] == 0.0:
                row_add = row + 1
                if dataframe.iloc[row_add, 1] != 0.0:
                    row+=1
                    return self.core_function(vehicle_name, row, end, dataframe, car_list)
                else:
                    row+=1
                    return self.filter_function(vehicle_name, row, end, dataframe, car_list)
            else:
                row+=1
                return self.filter_function(vehicle_name, row, end, dataframe, car_list)

        
    def core_function(self,vehicle_name, row, end, dataframe, car_list):
        """this function calculates the acceleration over a 3 second window"""
        temp_list = []
        timestamp = dataframe.iat[row,0]
        for i in range(3):
            temp_list.append(dataframe.iat[row,1])
            row+=1
        average_km = (reduce((lambda x, y: x+y), temp_list) / 3) 
        car_list.append({'timestamp': timestamp, 'value': average_km, 'vehicle': vehicle_name})
        row+=1
        return self.filter_function(vehicle_name, row, end, dataframe, car_list)

###########################################################################################################################
####This part of the class acceleration is my coded version of the Local Outlier Factor ###################################
###########################################################################################################################
    
    
    def selecting_k_distance_function(self, dataframe, k, car):
        """Main function under which all other function for determining the LOF factor is placed"""
        #determines the length of the dataframe
        row_dataframe = dataframe['timestamp'].count()
        k_value = k + 1
        dataframe= dataframe.round(1)
        #counter which will be used as x coordinate
        dataframe['counter'] = range(len(car1.dataframe_1))
        #unique id with x and y coordinates
        dataframe['unique_id'] = list(zip(dataframe['counter'], dataframe['value']))
        column_layout = ['unique_id']
        column_values = []
        observation_values = []
        #builds the appropriate dataframe size according to the inputs given
        for i in range(1, k_value):
            dataframe['value_k' + str(i)] = 1000
            column_layout.append('value_k' + str(i))
            column_values.append('value_k' + str(i))
            dataframe['observation_k' + str(i)] = -1
            column_layout.append('observation_k' + str(i))
            observation_values.append('observation_k' + str(i))
        dataframe = dataframe[column_layout]
        #calculating the Local reach density
        for i in range(1, k_value):
            self.all_columns_function(0,dataframe, i)
        dataframe['sum for LRD'] =  dataframe[column_values].sum(axis=1)
        dataframe['LRD'] = 1/(dataframe['sum for LRD']/k)
        
        observations =[]
        # not sure to be happy or worried about this fact that i can do a for-loop and listcomprehension simultaneously
        # reason for this choice: a list comprehension does not change data once '=' is reached, at least that is what my debugger insinuates 
        for i in dataframe[observation_values]:
            observations.append('check_'+ i)
            dataframe['check_'+ i] = [dataframe.loc[dataframe.loc[j,i], 'LRD'] for j in range(row_dataframe)]
        dataframe['LOF'] = (dataframe[observations].sum(axis=1)/k)/dataframe.loc[:,'LRD']
        
        
        outliers_dataframe = dataframe[dataframe['LOF'] > 1.5]
        outliers_counted_rows = len(dataframe[dataframe['LOF'] > 1.5])

        print('With the k being %s ' %k)
        print('the total rows of OUTLIERS : %s' %outliers_counted_rows)
       
        
       
    
    
    
    
    def all_columns_function(self, base_row,dataframe, k):
        """this function goes composes all the necessary columns"""
        row_dataframe = (dataframe['unique_id'].count() - 1) 
        if base_row > row_dataframe:
            return
        row_list_seq = list(range((base_row+1), (row_dataframe + 1)))
        row_list_seq.extend(range(base_row))
        for i in row_list_seq:
            self.min_function(base_row,i ,row_dataframe, dataframe, k)   
        base_row +=1
        return self.all_columns_function(base_row,dataframe, k)       
        
            
    
    def min_function(self,base_row, next_row,row_dataframe,dataframe, k):
        """This function together with f(euclides_function) determines the lowest euclidean value and places it in the correc
            column"""
        if k > 1:
            other_column_value = dataframe.loc[base_row,('value_k' + str(k-1))]
            current_value = dataframe.loc[base_row, ('value_k' + str(k))]
            temp_value = self.euclides_function(base_row,next_row, row_dataframe, dataframe)
            if (temp_value < current_value and temp_value > other_column_value):
                dataframe.loc[base_row,('value_k' + str(k))] = temp_value
                coordinate_of_value = dataframe.iloc[next_row, 0]
                coordinate_column = 'observation_k' + str(k)
                dataframe.set_value(base_row,coordinate_column,coordinate_of_value[0])
            else:
                pass
        else:
            current_value = dataframe.loc[base_row, ('value_k' + str(k))]
            temp_value = self.euclides_function(base_row,next_row, row_dataframe, dataframe)
            if temp_value < current_value:
                dataframe.loc[base_row,('value_k' + str(k))] = temp_value
                coordinate_of_value = dataframe.iloc[next_row, 0]
                coordinate_column = 'observation_k' + str(k)
                dataframe.set_value(base_row,coordinate_column,coordinate_of_value[0])
            else:
                pass      
    
    
      #the formula to calculate the euclidean distance between two point (as the crows fly)
    def euclides_function (self, base_row, next_row, row_dataframe,dataframe):
        """calculates either the euclidean value or the straight line depending on the circumstance"""
        tuple_base = dataframe.iloc[base_row,0]
        tuple_next = dataframe.iloc[next_row,0]  
        
        if tuple_base[1] == tuple_next[1]:
            euclide_distance = abs(tuple_base[0]-tuple_next[0])
            return euclide_distance
        else:
            euclide_distance = math.sqrt(math.pow((tuple_base[0] - tuple_next[0]),2) + 
                                         math.pow((tuple_base[1] - tuple_next[1]),2))
            return euclide_distance
    

In [3]:
car1 = acceleration()
#car2 = acceleration()
#car3 = acceleration()
#car4 = acceleration()
#car5 = acceleration()
#car6 = acceleration()
#car7 = acceleration()
car1.dataframe_1   = pd.DataFrame(car1.looping('car1', 0))
#car2.dataframe_1   = pd.DataFrame(car2.looping('car2', 0))
#car3.dataframe_1   = pd.DataFrame(car3.looping('car3', 0))
#car4.dataframe_1   = pd.DataFrame(car4.looping('car4', 0))
#car5.dataframe_1   = pd.DataFrame(car5.looping('car5', 0))
#car6.dataframe_1   = pd.DataFrame(car6.looping('car6', 0))
#car7.dataframe_1   = pd.DataFrame(car7.looping('car7', 0))

Requesting query... ok.
Job ID: job_0YemM68NaJrm2zH6RpGxz7SCGxvv
Query running...
Query done.
Processed: 47.5 MB
Standard price: $0.00 USD

Retrieving results...
Got 11000 rows.

Total time taken 2.12 s.
Finished at 2017-11-14 11:08:53.
Requesting query... ok.
Job ID: job_JIxiHTwu19MifgPNAOXlSQ5jIOpz
Query running...
Query done.
Processed: 47.5 MB
Standard price: $0.00 USD

Retrieving results...
Got 11000 rows.

Total time taken 2.22 s.
Finished at 2017-11-14 11:08:56.


From this histograph you can see that there are some probable outliers. An acceloration of above 40 km/h seems high. To make sure we cover all the basics lets first check the data.

In [ ]:
car1.datafram_validation = car1.dataframe_1[(car1.dataframe_1['value'] > 35)]
car1.datafram_validation

In [ ]:
print(pd.read_gbq("select * from [headingon_params.speed_obd]  where timestamp > '2016-10-12 18:18:19' and vehicle = 'car1' and timestamp < '2016-10-12 18:18:26' ", projectid))


print(pd.read_gbq("select * from [headingon_params.speed_obd]  where timestamp > '2016-10-16 13:31:28' and vehicle = 'car1' and timestamp < '2016-10-16 13:31:35' ", projectid))

in both cases a direct database query shows that the values computed are correct. for the sake of practice we will assume that this could actual be an outlier. What i mean is that if we had more data there could be multiple points this high, even by crossreffencing other data like altitude we could find valid explinations for this high accelaration. However,  We will focus on using a statistical learning algorithm to figure out which datapoints are *'inliers'* and which are *'outliers'*. First let as view the data from an different perspective. 

In [ ]:
car1.dataframe_1['counter'] = range(len(car1.dataframe_1))


car1.dataframe_1.plot.scatter(x='counter', y='value')


# algorithm selection

obvious my experience in ML is still growing thus it seems best to discuss some algorithms that I came accross and discuss which one I think is best suited to this particular situation.

So lets start with the actual goal we want to achieve:
We have a bunch of observation of average acceloration. There are some that seem to be clear *outliers* but there could be others which are not as clear. So what I am looking for is an algorithm which compute which oberservation are *'inliers'* and which are *'outliers'*. 

our data set is:
* non streaming data (due to data limitation)
* cannot be categorised as time series data

Therefore, i believe i should look into the two main Density-Based Anomaly detection 

##  K - nearest neigboor(KNN):
This algorithm uses the following basic logic: 'if a observation is closer to the *inliers* then to the *outliers* then it is probably an *inliers* and it will be treated as such'. The trick here is that you tell the algorithm how many of the closes point (k) you use to make this decision. For instance, if **k** is 3 that means that the nearest three points of that particulair observation will be taken and if the majority is *inliers* than that observation is classified as an *inlier* and vice versa. 

However, this algo supposes that you have at least 2 identifyable groups, in other words you know for sure which are *outliers* and which are not.Though in this example we assume that the two most extreme cases are outliers, i doubt it would be that clear-cut if I could import more data points (which is problematic due to the data limit in notebook). So I want an algorithm that defines the outliers essential by itself.
Another important concideration of the algorithm is the number of **k** you should use. there are several *rule-of-thumb* conciderations out there but I have not found a clear cut method. If possible I would like to take this guess element out as well but not sure how realistic this is.

## Local Outlier Factor(LOF):
The major difference between **knn** as i can see it is that the **lof**, is that it calculates both local and global clusters and their *outliers*. So what does that mean? If we look at the graph than the 2 points in the top are an example of global outliers, since they are far outside of the cluster bellow. But let us assume that we can add more datapoints and several of these are much closer to the *outliers* and thus they begin to form a cluster as well. So there are now 2 clusters. In the **knn** algorithm the smaller newer cluster would certainly be seen as a cluster of *outliers* while in the **lof** algorithm a part of the cluster could be seen as *inliers*, depending on the k-distance (the distance of k nearest neigboor).
It is this algorithm that i adopt to get to the question of which outliers I need to remove. then I can calculate the mean of the data and do the same for all 7 cars. 


### finding the k_distance:

coding objectives:

First we need to code something that will search for its k times closes neigboor. Since I could not find any convincing methode to calculate the appropriate k, the function should be able to calculate different k's on command.  
We also need to take care that points that are equal height are calculated differently than point that have a diagonal line connecting both points. 

### reachability-distance:

coding objectives:

Now we get into a less clear cut part of the algorithm. Called the *reachanility-distance*. This is the formula:

***Max(k_dist(B), dist(A,B))***

But according to how I read it this will always result in the same Max. I tried stackoverflow but the only entry I could find had the same question but no understandable answer:

https://stats.stackexchange.com/questions/112331/is-my-understanding-of-how-to-calculate-the-reachability-distance-in-local-outli

To not get stuck and also not try to attempt to code something that I don't fully grasp I have opted for an alternative route. This means that I will code the  *LRD, Local Reachability Distance* which takes the average of all the k's for that specific point and divides them with 1. what does that mean?

* for each observation the program will compute the k nearest neigbors. 
* create a new column which take all the k nearest neigbors of that observation averages them and divides them by 1, which gives us the Local Reachability Distance
* we then calculate the LRD of all the nearest neigbors, this does the following: we have the LRD of the actual observation but we want to compare this LRD with the LRD of the nearest neigbors of the observation, this allows us to answer the a very fundamental question namely: *'Are the nearest neigbors of the observation also the nearest neigbors of the observation '*. If i am the outlier then the people who I concider best friends won't concider me as their best friends because I am the outlier. This element makes it possible for the algorithm to determine the difference between local outliers as opposed to only global outliers. 
* the last column calculates the LOF (local outlier factor) which has a clear cutoff point of 1,5.



### which k to use:

In [4]:
car1.selecting_k_distance_function(car1.dataframe_1, 3, 'car1')

With the k being 3 
the total rows of OUTLIERS : 17


In [5]:
car1.selecting_k_distance_function(car1.dataframe_1, 4, 'car1')

With the k being 4 
the total rows of OUTLIERS : 16


In [6]:
car1.selecting_k_distance_function(car1.dataframe_1, 5, 'car1')

With the k being 5 
the total rows of OUTLIERS : 15


In [7]:
car1.selecting_k_distance_function(car1.dataframe_1, 6, 'car1')

With the k being 6 
the total rows of OUTLIERS : 14


In [8]:
car1.selecting_k_distance_function(car1.dataframe_1, 7, 'car1')

With the k being 7 
the total rows of OUTLIERS : 13


In [9]:
car1.selecting_k_distance_function(car1.dataframe_1, 8, 'car1')

With the k being 8 
the total rows of OUTLIERS : 13


In [10]:
car1.selecting_k_distance_function(car1.dataframe_1, 9, 'car1')

With the k being 9 
the total rows of OUTLIERS : 13


## conclusion which k:

k=7 seems to be the best given the data

# recap and improvements:

The LOF algorithm is thus far the most non trivial algo I have encoded. And I found the process very engaging and rewarding.

What is less rewarding is the problems I have encountered both with the datalimits (have others encountered this?!).
And in order to develop the vizuals I encountered problems, once again, which leads me to the conclusion that i need guidance on this, since just coding like i now do apparently is not the answer.